## UGCA Group Assignment 3

# Group Members
1. Chetna Singhal
2. David Kinman
3. Prajval Gupta
4. Subhayu Chakravarty
5. Whitt Hyde

In [98]:
from selenium import webdriver
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

### PART A - Extracing Beer Reviews

In [4]:
### Scraping beer reviewa via Selenium ###

driver = webdriver.Chrome(executable_path='/Users/livestrong19/Downloads/chromedriver')
driver.get('https://www.beeradvocate.com/beer/top-rated/')

### Fetch names of 250 beers and URL link
beer_dict={}
for i in range(2,252):
    beer_path=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr['+str(i)+']/td[2]/a')
    beer_name=driver.find_element_by_xpath('//div[@id="ba-content"]/table/tbody/tr['+str(i)+']/td[2]/a/b').text
    beer_dict[beer_name]=beer_path.get_attribute('href')
driver.close()

In [5]:
# Extraction Round 1
### Fetch first 25 reviews for each of the 250 beers

beer_ratings = pd.DataFrame(columns = ['Product Name','User Rating','Product Review']) 

try:
    for key,value in beer_dict.items():
        driver = webdriver.Chrome('/Users/livestrong19/Downloads/chromedriver')
        #driver = webdriver.Chrome('/Users/prajvalgupta/Downloads/chromedriver 2')
        driver.get(value)
        reviews = driver.find_elements_by_xpath('//div[@id="rating_fullview_content_2"]')
        for review in reviews:
            review_text = review.text
            rating = review.find_element_by_xpath('.//span[1]').text
            extra1 = review.find_element_by_xpath('.//span[2]').text
            extra2 = review.find_element_by_xpath('.//span[3]').text
            extra3 = review.find_element_by_xpath('.//span[4]').text
            extra4 = review.find_element_by_xpath('.//span[5]').text
            extra5 = review.find_element_by_xpath('.//div').text
            
            if extra1:
                review_text = review_text.replace(extra1,"")
            if extra2:
                review_text = review_text.replace(extra2,"")
            if extra3:
                review_text = review_text.replace(extra3,"")
            if extra4:
                review_text = review_text.replace(extra4,"")
            if extra5:
                review_text = review_text.replace(extra5,"")

            beer_ratings.loc[len(beer_ratings)] = [key,rating,review_text]

        driver.close()
except:
    pass

In [8]:
beer_ratings

,Product Name,User Rating,Product Review
0,Kentucky Brunch Brand Stout,4.8,4.8 rDev \n\n\n2016 Silver Wax. Aroma has whi...
1,Kentucky Brunch Brand Stout,4.74,4.74 rDev \n\n\nThe beer pours Pitch Black wi...
2,Kentucky Brunch Brand Stout,4.68,4.68 rDev \n\n\nProbably the smoothest beer I...
3,Kentucky Brunch Brand Stout,5,"5 rDev \n\n\nDark black, very thick, a little..."
4,Kentucky Brunch Brand Stout,4.97,4.97 rDev \n\n\nPoured black as ink with thin...
...,...,...,...
4826,Black & Wild,3.44,3.44 rDev \n\n\nNot anything I want to try ag...
4827,Black & Wild,4.66,4.66 rDev \n\n\n8oz pour on draft at the brew...
4828,Black & Wild,4.38,4.38 rDev \n\n\nWas surprised that I got to t...
4829,Black & Wild,4.74,4.74 rDev \n\n\nOne of the most unique wild's...


In [60]:
beer_ratings['Product Name'].value_counts()

Double Stack                                           25
Parabola                                               25
Barrel Aged Imperial German Chocolate Cupcake Stout    25
Clover                                                 25
Nectarine Premiere                                     25
                                                       ..
4th Anniversary                                        22
Anabasis                                               20
It Was All A Dream                                     20
Chemtrailmix                                           17
Black & Wild                                            5
Name: Product Name, Length: 195, dtype: int64

In [19]:
beer1 = list(beer_dict.keys())
beer2 = list(beer_ratings['Product Name'].unique())
print(len(beer1),len(beer2))
beer_missing = list(set(beer1)-set(beer2))

In [53]:
# Extraction Round 2
### Fetch reviews for missing 55 beers

beer_ratings2 = pd.DataFrame(columns = ['Product Name','User Rating','Product Review']) 


for key,value in beer_dict.items():
    if key in beer_missing:
        driver = webdriver.Chrome('/Users/livestrong19/Downloads/chromedriver')
        driver.get(value+"?view=beer&sort=&start=25")
        reviews = driver.find_elements_by_xpath('//div[@id="rating_fullview_content_2"]')
        
        for review in reviews:
            review_text = review.text
            
            rating = review.find_element_by_xpath('.//span[1]').text
            extra1 = review.find_element_by_xpath('.//span[2]').text
            extra2 = review.find_element_by_xpath('.//span[3]').text
            extra3 = review.find_element_by_xpath('.//span[4]').text
            try:
                extra4 = review.find_element_by_xpath('.//span[5]').text
                if extra4:
                    review_text = review_text.replace(extra4,"")
            except:
                pass

            extra5 = review.find_element_by_xpath('.//div').text

            if extra1:
                review_text = review_text.replace(extra1,"")
            if extra2:
                review_text = review_text.replace(extra2,"")
            if extra3:
                review_text = review_text.replace(extra3,"")

            if extra5:
                review_text = review_text.replace(extra5,"")
            
            beer_ratings2.loc[len(beer_ratings2)] = [key,rating,review_text]

        driver.close()

In [54]:
beer_ratings2

,Product Name,User Rating,Product Review
0,Ten FIDY - Bourbon Barrel Aged,4.38,4.38 rDev \n\n\n19.2oz can into a goblet. Pac...
1,Ten FIDY - Bourbon Barrel Aged,4,"4 rDev \n\n\nVintage optics - oil black, quar..."
2,Ten FIDY - Bourbon Barrel Aged,4.58,4.58 rDev \n\n\nPours almost black with a kha...
3,Ten FIDY - Bourbon Barrel Aged,4.37,4.37 rDev \n\n\n2017 Vintage GVBX Draught 112...
4,Ten FIDY - Bourbon Barrel Aged,4.43,"4.43 rDev \n\n\nOn tap at Beer Shop, NYC.\nPo..."
...,...,...,...
1163,Sure Bet,4.74,"4.74 rDev \n\n\nBomber. Pours opaque orange, ..."
1164,Sure Bet,4.07,"4.07 rDev \n\n\nNot their best, but another s..."
1165,Sure Bet,4.48,4.48 rDev \n\n\nLooks fantastic - hazy orange...
1166,Sure Bet,4.46,"4.46 rDev \n\n\nL - foggy copper orange, nice..."


In [66]:
beer_ratings_final = beer_ratings.append(beer_ratings2, ignore_index = True)
beer_ratings_final.to_csv("beer_reviews.csv")

### PART B - Selecting 3 beer attributes using word frequency analysis

In [78]:
# Load beer reviews into a dataframe
df = pd.read_csv("beer_reviews.csv",usecols=["Product Name","User Rating","Product Review"])

# Remove newline from each post & drop rows with null values
df = df.replace('\n','', regex=True)
df = df.replace('rDev','', regex=True)
df = df.dropna()
df

,Product Name,User Rating,Product Review
0,Kentucky Brunch Brand Stout,4.80,"4.8 2016 Silver Wax. Aroma has whiskey, mapl..."
1,Kentucky Brunch Brand Stout,4.74,4.74 The beer pours Pitch Black with a froth...
2,Kentucky Brunch Brand Stout,4.68,4.68 Probably the smoothest beer I have ever...
3,Kentucky Brunch Brand Stout,5.00,"5 Dark black, very thick, a little bit of ta..."
4,Kentucky Brunch Brand Stout,4.97,4.97 Poured black as ink with thin ruby edge...
...,...,...,...
5994,Sure Bet,4.74,"4.74 Bomber. Pours opaque orange, two finger..."
5995,Sure Bet,4.07,"4.07 Not their best, but another solid DIPA ..."
5996,Sure Bet,4.48,4.48 Looks fantastic - hazy orange with a 2 ...
5997,Sure Bet,4.46,"4.46 L - foggy copper orange, nice head, bea..."


In [80]:
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

def review_cleaner(text):
    
    # Remove numbers
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    
    lower_case = letters_only.lower()
    
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [89]:
clean_reviews = []

for i in range(df.shape[0]):
    clean_reviews.append(review_cleaner(df['Product Review'][i]))
    
clean_reviews = pd.DataFrame(clean_reviews)
df["Clean Product Review"]=clean_reviews
df

,Product Name,User Rating,Product Review,Clean Product Review
0,Kentucky Brunch Brand Stout,4.80,"4.8 2016 Silver Wax. Aroma has whiskey, mapl...",silver wax aroma has whiskey maple toffee and ...
1,Kentucky Brunch Brand Stout,4.74,4.74 The beer pours Pitch Black with a froth...,the beer pours pitch black with a frothy tan h...
2,Kentucky Brunch Brand Stout,4.68,4.68 Probably the smoothest beer I have ever...,probably the smoothest beer i have ever had sm...
3,Kentucky Brunch Brand Stout,5.00,"5 Dark black, very thick, a little bit of ta...",dark black very thick a little bit of tan head...
4,Kentucky Brunch Brand Stout,4.97,4.97 Poured black as ink with thin ruby edge...,poured black as ink with thin ruby edges at de...
...,...,...,...,...
5994,Sure Bet,4.74,"4.74 Bomber. Pours opaque orange, two finger...",bomber pours opaque orange two finger white he...
5995,Sure Bet,4.07,"4.07 Not their best, but another solid DIPA ...",not their best but another solid dipa offering...
5996,Sure Bet,4.48,4.48 Looks fantastic - hazy orange with a 2 ...,looks fantastic hazy orange with a finger whit...
5997,Sure Bet,4.46,"4.46 L - foggy copper orange, nice head, bea...",l foggy copper orange nice head beautiful laci...


In [100]:
# Remove all punctuation characters from each tweet and convert text to lowercase for efficient frequency counting
# punctuation includes !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~

def remove_punctuations(item):
    for p in punctuation:
        item = item.strip().replace(p,'')
    return item

# Function to remove stopwords
def remove_stopwords(s):
    stop_words = set(stopwords.words('english'))
    return [w for w in s if not w in stop_words] 

In [105]:
# Tokenize the posts
df['Tokens'] = df["Clean Product Review"].apply(word_tokenize).apply(list)
# df['Tokens'] = df['Tokens'].apply(remove_punctuations)
df['Tokens'] = df['Tokens'].apply(remove_stopwords)
df

,Product Name,User Rating,Product Review,Clean Product Review,Tokens
0,Kentucky Brunch Brand Stout,4.80,"4.8 2016 Silver Wax. Aroma has whiskey, mapl...",silver wax aroma has whiskey maple toffee and ...,"[silver, wax, aroma, whiskey, maple, toffee, u..."
1,Kentucky Brunch Brand Stout,4.74,4.74 The beer pours Pitch Black with a froth...,the beer pours pitch black with a frothy tan h...,"[beer, pours, pitch, black, frothy, tan, head,..."
2,Kentucky Brunch Brand Stout,4.68,4.68 Probably the smoothest beer I have ever...,probably the smoothest beer i have ever had sm...,"[probably, smoothest, beer, ever, smelled, bet..."
3,Kentucky Brunch Brand Stout,5.00,"5 Dark black, very thick, a little bit of ta...",dark black very thick a little bit of tan head...,"[dark, black, thick, little, bit, tan, head, s..."
4,Kentucky Brunch Brand Stout,4.97,4.97 Poured black as ink with thin ruby edge...,poured black as ink with thin ruby edges at de...,"[poured, black, ink, thin, ruby, edges, degree..."
...,...,...,...,...,...
5994,Sure Bet,4.74,"4.74 Bomber. Pours opaque orange, two finger...",bomber pours opaque orange two finger white he...,"[bomber, pours, opaque, orange, two, finger, w..."
5995,Sure Bet,4.07,"4.07 Not their best, but another solid DIPA ...",not their best but another solid dipa offering...,"[best, another, solid, dipa, offering, pipewor..."
5996,Sure Bet,4.48,4.48 Looks fantastic - hazy orange with a 2 ...,looks fantastic hazy orange with a finger whit...,"[looks, fantastic, hazy, orange, finger, white..."
5997,Sure Bet,4.46,"4.46 L - foggy copper orange, nice head, bea...",l foggy copper orange nice head beautiful laci...,"[l, foggy, copper, orange, nice, head, beautif..."


In [141]:
count = []

# Count the frequency of the words
for i in range(len(df["Tokens"])):
    count += df.iloc[i]["Tokens"]

count = [x.lower() for x in count]
word_freq = nltk.FreqDist(count)

# 50 most frequent words
top_words = word_freq.most_common(500)
top_words

[('beer', 4981),
 ('head', 3472),
 ('taste', 2844),
 ('chocolate', 2610),
 ('dark', 2576),
 ('like', 2537),
 ('sweet', 2247),
 ('one', 2171),
 ('nice', 2082),
 ('coffee', 2079),
 ('bourbon', 2062),
 ('light', 1955),
 ('orange', 1906),
 ('well', 1860),
 ('good', 1849),
 ('nose', 1846),
 ('notes', 1825),
 ('vanilla', 1787),
 ('medium', 1723),
 ('finish', 1701),
 ('aroma', 1660),
 ('flavor', 1646),
 ('pours', 1636),
 ('carbonation', 1616),
 ('bit', 1611),
 ('fruit', 1591),
 ('white', 1579),
 ('body', 1507),
 ('bottle', 1504),
 ('little', 1497),
 ('citrus', 1460),
 ('great', 1453),
 ('black', 1450),
 ('really', 1446),
 ('barrel', 1423),
 ('mouthfeel', 1422),
 ('smooth', 1394),
 ('overall', 1343),
 ('flavors', 1340),
 ('thick', 1316),
 ('oak', 1298),
 ('lacing', 1258),
 ('glass', 1248),
 ('brown', 1224),
 ('color', 1193),
 ('feel', 1186),
 ('malt', 1120),
 ('creamy', 1091),
 ('smell', 1085),
 ('bitterness', 1063),
 ('poured', 1044),
 ('much', 1039),
 ('dry', 1008),
 ('sweetness', 1001),
 ('

In [149]:
word_list = list(sum(top_words, ()))
new_word_list = {word_list[i]: word_list[i + 1] for i in range(0, len(word_list), 2)} 
new_word_list

{'beer': 4981,
 'head': 3472,
 'taste': 2844,
 'chocolate': 2610,
 'dark': 2576,
 'like': 2537,
 'sweet': 2247,
 'one': 2171,
 'nice': 2082,
 'coffee': 2079,
 'bourbon': 2062,
 'light': 1955,
 'orange': 1906,
 'well': 1860,
 'good': 1849,
 'nose': 1846,
 'notes': 1825,
 'vanilla': 1787,
 'medium': 1723,
 'finish': 1701,
 'aroma': 1660,
 'flavor': 1646,
 'pours': 1636,
 'carbonation': 1616,
 'bit': 1611,
 'fruit': 1591,
 'white': 1579,
 'body': 1507,
 'bottle': 1504,
 'little': 1497,
 'citrus': 1460,
 'great': 1453,
 'black': 1450,
 'really': 1446,
 'barrel': 1423,
 'mouthfeel': 1422,
 'smooth': 1394,
 'overall': 1343,
 'flavors': 1340,
 'thick': 1316,
 'oak': 1298,
 'lacing': 1258,
 'glass': 1248,
 'brown': 1224,
 'color': 1193,
 'feel': 1186,
 'malt': 1120,
 'creamy': 1091,
 'smell': 1085,
 'bitterness': 1063,
 'poured': 1044,
 'much': 1039,
 'dry': 1008,
 'sweetness': 1001,
 'stout': 954,
 'hops': 940,
 'caramel': 938,
 'grapefruit': 915,
 'full': 905,
 'big': 878,
 'tropical': 868,


In [160]:
# Finding top 3 attributes basis word frequency analysis

beer_attribute_list = [["aggressive",0],["balanced",0],["complex",0],["crisp",0],["fruity",0],["hoppy",0],["malty",0],["robust",0]]
beer_attribute_list = pd.DataFrame(beer_attribute_list)
beer_attribute_list.columns = ["Attribute","Word_Frequency"]
beer_attribute_list

,Attribute,Word_Frequency
0,aggressive,0
1,balanced,0
2,complex,0
3,crisp,0
4,fruity,0
5,hoppy,0
6,malty,0
7,robust,0


In [161]:
for i in range(len(beer_attribute_list["Attribute"])):
    for key, value in new_word_list.items():
        if beer_attribute_list["Attribute"][i] in key:
            beer_attribute_list["Word_Frequency"].iloc[i] = value

beer_attribute_list = beer_attribute_list.sort_values(by=['Word_Frequency'],ascending=False)
beer_attribute_list

,Attribute,Word_Frequency
1,balanced,832
4,fruity,530
2,complex,312
3,crisp,297
5,hoppy,275
6,malty,193
0,aggressive,0
7,robust,0


In [188]:
# Storing top 3 attributes in file
beer_attribute_top3_list = beer_attribute_list.iloc[0:3,0]
beer_attribute_top3_list.to_csv("user_beer_attributes.csv",index=False, header="Attributes")

### PART C - Similarity analysis between Beer Attributes and Beer Reviews

### PART D - Sentiment Analysis of Beer Reviews

### PART E - Top 3 beer recommendations - Method I

### PART F - Top 3 beer recommendations - Method II